In [1]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from datetime import datetime
import io
import itertools
from packaging import version
from six.moves import range

import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics 

In [2]:
import modelBuilder 

In [3]:
from dataLoader import getDataset

In [4]:
%load_ext tensorboard

In [5]:
workingdir = "/home/rene/cla_dataset/cam0_preprocessed/"

In [6]:
ds_train, ds_valid = getDataset(workingdir)

In [69]:
# Clear out any prior log data.
# Sets up a timestamped log directory.
logdir = "logs/train_data/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(logdir)

    
    
with file_writer.as_default():
    tf.summary.image("GT", l/2, step=0)



NameError: name 'l' is not defined

In [8]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 4886), started 0:24:34 ago. (Use '!kill 4886' to kill it.)

In [71]:
pred = None
pred_raw = None
def print_valid_images(epoch, logs):
    global pred_raw
    global pred
    # Use the model to predict the values from the validation dataset.
    for img, label in ds_valid.take(1):
        test_pred_raw = mdl.predict(img)
        pred_raw = test_pred_raw
        pred_mask = tf.argmax(test_pred_raw, axis=3)
        pred_mask = pred_mask[..., tf.newaxis]

        with file_writer.as_default():
            tf.summary.image("Valid_img", img, step = 0)

        with file_writer.as_default():
            tf.summary.image("Valid_gt", label/2, step = 0)
            
        with file_writer.as_default():
            tf.summary.image("Valid_predict", pred_mask/2, step = 0)
        
        print(pred_mask.shape)
        pred = pred_mask
# Define the per-epoch callback.
cm_callback = keras.callbacks.LambdaCallback(on_epoch_end=print_valid_images)


In [67]:
mdl = modelBuilder.getModel()
mdl.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs", histogram_freq=1)


In [72]:
model_history = mdl.fit(ds_train, epochs=2,steps_per_epoch=1, callbacks = [tensorboard_callback, cm_callback])

Epoch 1/2
1/1 [==============================] - 1s 519ms/step - loss: 1.0940 - accuracy: 0.3582
Epoch 2/2
1/1 [==============================] - 0s 427ms/step - loss: 1.1012 - accuracy: 0.3077


In [250]:
def create_mask(pred_mask):
    pred_mask = tf.argmax(pred_mask, axis=-1)
    pred_mask = pred_mask[..., tf.newaxis]
    return pred_mask[0]


In [256]:
def show_predictions(dataset=None, num=1):
    if dataset:
        for image, mask in dataset.take(num):
            pred_mask = model.predict(image)
            display([image[0], mask[0], create_mask(pred_mask)])
    else:
        display([sample_image, sample_mask,
                 create_mask(model.predict(sample_image[tf.newaxis, ...]))])


In [287]:
class DisplayCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        #clear_output(wait=True)
        show_predictions(dataset)
        print ('\nSample Prediction after epoch {}\n'.format(epoch+1))
